# **Agentic AI Audio**

In [ ]:
#Mount Drive & unzip

from google.colab import drive
drive.mount('/content/drive')

!ls -lh "/content/drive/MyDrive/"

import os

# Make sure Deepfake_audio exists, else unzip
if not os.path.exists("/content/drive/MyDrive/Deepfake_audio"):
    !unzip -q "/content/drive/MyDrive/Deepfake_audio.zip" -d "/content/drive/MyDrive/Deepfake_audio"
else:
    print("Already extracted ✅")


Mounted at /content/drive
ls: '/content/drive/MyDrive/MUFG SIH': No such file or directory
ls: /content/drive/MyDrive/audio_results_batches: No such file or directory
ls: /content/drive/MyDrive/Deepfake_audio: No such file or directory
total 17G
lrw------- 1 root root    0 Sep 16 13:56  audio_results_batches -> /content/drive/.shortcut-targets-by-id/1jrzVTsF60OQ3lj2j632BW99dA8L_G-Fp/audio_results_batches
drwx------ 2 root root 4.0K Aug 16 13:33 'Colab Notebooks'
lrw------- 1 root root    0 Sep 16 13:56  Deepfake_audio -> /content/drive/.shortcut-targets-by-id/1wGKV0HToLh6AtQjUAJsSD2gkVL4onNIB/Deepfake_audio
-rw------- 1 root root  17G Sep 13 21:37  Deepfake_audio.zip
lrw------- 1 root root    0 Sep 15 19:44 'MUFG SIH' -> '/content/drive/.shortcut-targets-by-id/1RKTAHMoiCvedxpEpKg4fl4mV16mXm7qQ/MUFG SIH'
Already extracted ✅


In [ ]:
#Install dependencies
!pip install torch torchaudio transformers librosa soundfile tqdm pyannote.audio
!pip install git+https://github.com/openai/whisper.git


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.8/897.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.5/828.5 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Filter audio files ≥2s

import glob
import soundfile as sf
from tqdm import tqdm

# Collect all wav files recursively
all_audio_files = glob.glob("/content/drive/MyDrive/Deepfake_audio/**/*.wav", recursive=True)

# Filter files by duration >= 2 seconds
audio_files = []
for f in tqdm(all_audio_files, desc="Filtering audio files"):
    try:
        f_info = sf.info(f)
        duration = f_info.frames / f_info.samplerate
        if duration >= 2.0:
            audio_files.append(f)
    except RuntimeError:
        print(f"Skipping {f} due to error")

print(f"Found {len(audio_files)} wav files ≥ 2s")
print("Example files:", audio_files[:5])


Filtering audio files: 100%|██████████| 14101/14101 [1:20:10<00:00,  2.93it/s]

Found 14101 wav files ≥ 2s
Example files: ['/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file1723.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file1818.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file2192.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file961.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file2018.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav']


In [ ]:
#Load pretrained models

import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import whisper
from pyannote.audio import Model, Inference

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Wav2Vec2
wav2vec_model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(wav2vec_model_name)
wav2vec_model = Wav2Vec2Model.from_pretrained(wav2vec_model_name).to(device)

# Whisper
whisper_model = whisper.load_model("small")

# Pyannote speaker embedding
pyannote_model = Model.from_pretrained(
    "pyannote/embedding",
    HF_TOKEN = "<your-secret-token>"

)
speaker_model = Inference(pyannote_model, window="whole")
speaker_model.to(device)


/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access publi

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/migration/migration.py:208: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--embedding/snapshots/4db4899737a38b2d618bbd74350915aa10293cb2/pytorch_model.bin`
/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/model.py:692: UserWarning: Model has been trained with a task-dependent loss function. Set 'strict' to False to load the model without its loss

Model was trained with pyannote.audio 0.0.1, yours is 3.4.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.8.0+cu126. Bad things might happen unless you revert torch to 1.x.


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--embedding/snapshots/4db4899737a38b2d618bbd74350915aa10293cb2/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.4.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.8.0+cu126. Bad things might happen unless you revert torch to 1.x.


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['loss_func.W']


In [ ]:
# Embedding extraction functions
import numpy as np
import torch
import librosa

def extract_wav2vec_embedding(file_path):
    audio, sr = librosa.load(file_path, sr=16000)
    input_values = processor(audio, sampling_rate=sr, return_tensors="pt").input_values.to(device)
    with torch.no_grad():
        embeddings = wav2vec_model(input_values).last_hidden_state
    return embeddings.mean(dim=1).squeeze().cpu().numpy()

def extract_whisper_embedding(file_path, max_len=512):
    result = whisper_model.transcribe(file_path)
    text = result["text"]
    text_vec = np.zeros(max_len)
    for i, c in enumerate(text[:max_len]):
        text_vec[i] = ord(c)
    return text_vec

def extract_speaker_embedding(file_path):
    emb = speaker_model(file_path)
    if not isinstance(emb, np.ndarray):
        emb = emb.squeeze()
        if torch.is_tensor(emb):
            emb = emb.cpu().numpy()
    return emb


In [ ]:
# Fusion embeddings & alerts
from scipy.spatial.distance import cdist
import json
import os
import numpy as np
from tqdm import tqdm

# Config
save_embeddings_path = "/content/drive/MyDrive/fusion_embeddings.npz"
save_alerts_path = "/content/drive/MyDrive/audio_alerts.jsonl"
threshold = 0.85
w_spoof, w_speaker, w_content = 0.5, 0.3, 0.2

# Optional reference speaker embedding
ref_speaker_file = None
ref_emb = extract_speaker_embedding(ref_speaker_file) if ref_speaker_file else None

fusion_embeddings = []
filenames = []
alerts = []

for file_path in tqdm(audio_files, desc="Processing audio files"):
    # Extract embeddings
    wav2vec_emb = extract_wav2vec_embedding(file_path)
    whisper_emb = extract_whisper_embedding(file_path)
    speaker_emb = extract_speaker_embedding(file_path)

    # Fuse embeddings
    fusion_emb = np.concatenate([wav2vec_emb, whisper_emb, speaker_emb])
    fusion_embeddings.append(fusion_emb)
    filenames.append(os.path.basename(file_path))

    # Spoof probability placeholder
    spoof_prob = 0.5  # Replace with real Wav2Vec2 spoof classifier

    # Speaker similarity
    speaker_sim = None
    if ref_emb is not None:
        speaker_sim = 1 - cdist([speaker_emb], [ref_emb], metric="cosine")[0,0]

    # Content similarity placeholder
    content_sim = 1.0

    # Fused score
    fused_score = w_spoof*spoof_prob + w_speaker*(speaker_sim or 0) + w_content*content_sim
    alert_flag = fused_score > threshold

    alerts.append({
        "file": os.path.basename(file_path),
        "fused_score": float(fused_score),
        "prediction": "spoof" if alert_flag else "clean",
        "speaker_similarity": float(speaker_sim) if speaker_sim is not None else None,
        "content_similarity": float(content_sim)
    })

# Save fusion embeddings
fusion_embeddings = np.array(fusion_embeddings)
np.savez(save_embeddings_path, embeddings=fusion_embeddings, filenames=filenames)
print(f"✅ Saved fusion embeddings to {save_embeddings_path}")

# Save alerts
with open(save_alerts_path, 'w') as f:
    for a in alerts:
        f.write(json.dumps(a)+'\n')
print(f"✅ Saved alerts to {save_alerts_path}")


Processing audio files:   0%|          | 0/14101 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
Processing audio files:   0%|          | 1/14101 [00:36<141:34:55, 36.15s/it]/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warni

KeyboardInterrupt: 

In [ ]:
import pandas as pd

csv_path = "/content/drive/MyDrive/MUFG SIH/processed_audio_mfcc.csv"  # (yours lives in MUFG SIH)
df = pd.read_csv(csv_path)

# Robust mapping (edit if needed)
SPOOF_FOLDERS = {"for-norm","for-rerec","for-2sec"}
df["label"] = df["subset"].apply(lambda s: "bonafide" if s == "for-original" else ("spoof" if s in SPOOF_FOLDERS else "unknown"))
df = df[df["label"] != "unknown"].copy()

mfcc_cols = [c for c in df.columns if c.startswith("mfcc_")]
print(df["label"].value_counts())

label
spoof       95209
bonafide    68725
Name: count, dtype: int64


In [ ]:
!pip -q install xgboost

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from xgboost import XGBClassifier
import joblib, os

X = df[mfcc_cols].values
y = (df["label"] == "spoof").astype(int).values   # 1=spoof, 0=bonafide

# Split
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale helps some models; XGB is tree-based (doesn't require it), but it’s fine to keep a scaler for consistency
scaler = StandardScaler().fit(Xtr)
Xtr_s = scaler.transform(Xtr)
Xte_s = scaler.transform(Xte)

# XGBoost baseline (tune these fast)
clf = XGBClassifier(
    n_estimators=400,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
    eval_metric="logloss"
)
clf.fit(Xtr_s, ytr)

proba = clf.predict_proba(Xte_s)[:,1]
pred = (proba >= 0.5).astype(int)

print(classification_report(yte, pred, target_names=["bonafide","spoof"]))
print("Confusion matrix:\n", confusion_matrix(yte, pred))
try:
    print("ROC-AUC:", roc_auc_score(yte, proba))
except Exception as e:
    print("ROC-AUC: NA", e)

# Save model bundle
bundle_dir = "/content/drive/MyDrive/MUFG SIH/models_audio"
os.makedirs(bundle_dir, exist_ok=True)
joblib.dump({"scaler": scaler, "model": clf, "mfcc_cols": mfcc_cols}, os.path.join(bundle_dir, "xgb_mfcc_spoof_detector.joblib"))
print("Saved:", os.path.join(bundle_dir, "xgb_mfcc_spoof_detector.joblib"))

              precision    recall  f1-score   support

    bonafide       0.92      0.94      0.93     13745
       spoof       0.95      0.94      0.95     19042

    accuracy                           0.94     32787
   macro avg       0.94      0.94      0.94     32787
weighted avg       0.94      0.94      0.94     32787

Confusion matrix:
 [[12878   867]
 [ 1086 17956]]
ROC-AUC: 0.985730169174006
Saved: /content/drive/MyDrive/MUFG SIH/models_audio/xgb_mfcc_spoof_detector.joblib


In [ ]:
import librosa, numpy as np, joblib

bundle = joblib.load("/content/drive/MyDrive/MUFG SIH/models_audio/xgb_mfcc_spoof_detector.joblib")
scaler = bundle["scaler"]; model = bundle["model"]; mfcc_cols = bundle["mfcc_cols"]

def mfcc_mean_vec(path, sr=16000, n_mfcc=13):
    y, sr = librosa.load(path, sr=sr, mono=True)
    if len(y) < sr: return None
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfcc.mean(axis=1)

def predict_audio_spoof(path):
    vec = mfcc_mean_vec(path)
    if vec is None: return {"ok": False, "reason": "too short"}
    X = vec.reshape(1, -1)
    Xs = scaler.transform(X)
    p = model.predict_proba(Xs)[0,1]
    return {"ok": True, "spoof_prob": float(p), "label": "spoof" if p>=0.5 else "bonafide"}

# Example:
# print(predict_audio_spoof("/content/deepfake_audio_dataset/for-norm/somefile.wav"))

In [ ]:
# PSEUDOCODE – after you clone a lip-sync repo (e.g., Wav2Lip)
#!git clone https://github.com/Rudrabha/Wav2Lip.git
#%cd Wav2Lip
#!pip install -r requirements.txt

def lipsync_score(video_path) -> float:
    """
    Return a scalar sync score. Implement using the repo’s inference utilities.
    Common approach: the provided `infer.py` outputs confidence scores; parse them.
    """
    # Example: run the inference script and parse stdout/json
    # score = run_and_parse(video_path)
    score = 0.84  # placeholder
    return float(score)

In [ ]:
# Install tesseract & deps
!apt-get -qq update
!apt-get -qq install tesseract-ocr
!pip -q install pytesseract opencv-python imagehash

import cv2, pytesseract, numpy as np, imagehash
from PIL import Image
import textwrap, os

def sample_video_frames(video_path, step=15):
    cap = cv2.VideoCapture(video_path)
    frames = []
    i = 0
    while True:
        ret, frame = cap.read()
        if not ret: break
        if i % step == 0:
            frames.append(frame)
        i += 1
    cap.release()
    return frames

def ocr_text_from_frame(frame_bgr):
    gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    # light denoise / threshold can help
    txt = pytesseract.image_to_string(gray)
    txt = txt.strip()
    return txt

def ocr_video(video_path, max_frames=20, step=15):
    frames = sample_video_frames(video_path, step=step)
    texts = []
    for f in frames[:max_frames]:
        t = ocr_text_from_frame(f)
        if t:
            texts.append(t)
    joined = "\n".join(texts)
    return joined

# suspicious keyword scan (customize)
SUSPICIOUS = ["urgent", "reset password", "verify account", "OTP", "bank", "winner", "click here"]

def ocr_suspicion_score(ocr_text: str) -> float:
    text_low = ocr_text.lower()
    hits = sum(1 for k in SUSPICIOUS if k in text_low)
    return min(1.0, hits / 5.0)

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
from skimage.metrics import structural_similarity as ssim

def frame_luma(frame_bgr):
    return cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)

def background_drift_score(video_path, max_pairs=25, step=30):
    frames = sample_video_frames(video_path, step=step)
    if len(frames) < 3:
        return 0.0
    # crude: compare every k-th pair
    drifts = []
    prev = frame_luma(frames[0])
    for f in frames[1:max_pairs]:
        cur = frame_luma(f)
        s = ssim(prev, cur)
        drifts.append(1.0 - s)  # larger => more drift
        prev = cur
    # robust aggregate
    return float(np.clip(np.median(drifts), 0.0, 1.0))

In [ ]:
import os, mimetypes, json, time
from dataclasses import dataclass, asdict

@dataclass
class ModalityResult:
    name: str
    score: float
    details: dict
    passed: bool

@dataclass
class Decision:
    final_label: str
    final_score: float
    rationale: str
    components: list

def detect_modality(path):
    mime, _ = mimetypes.guess_type(path)
    if not mime:
        ext = os.path.splitext(path)[1].lower()
        if ext in [".wav",".mp3",".flac",".m4a",".ogg",".aac",".wma"]:
            return "audio"
        if ext in [".mp4",".mov",".mkv",".avi",".webm"]:
            return "video"
        return "unknown"
    if mime.startswith("audio"): return "audio"
    if mime.startswith("video"): return "video"
    return "unknown"

In [ ]:
def audio_module(path):
    out = predict_audio_spoof(path)  # from earlier
    if not out["ok"]:
        return ModalityResult("audio_spoof", 0.5, {"reason": out.get("reason")}, False)
    # define score as spoof_prob (higher => more spoofy)
    score = float(out["spoof_prob"])
    passed = (score < 0.5)  # pass == likely bona fide
    return ModalityResult("audio_spoof", score, {"label": out["label"]}, passed)

In [ ]:
def video_module(path):
    lip = lipsync_score(path)                  # 0..1, higher better sync
    ocr_txt = ocr_video(path)
    ocr_susp = ocr_suspicion_score(ocr_txt)    # 0..1, higher = more suspicious
    drift = background_drift_score(path)       # 0..1, higher = more drift

    # Convert to "spoofiness"
    # If lipsync is LOW -> more spoofy; map spoofiness = 1 - lip
    spoil_lip = float(1.0 - lip)
    # OCR suspicious words already 0..1
    spoil_ocr = float(ocr_susp)
    # Background drift already 0..1
    spoil_bg = float(drift)

    # Fuse (weights adjustable)
    weights = np.array([0.5, 0.3, 0.2])
    vec = np.array([spoil_lip, spoil_ocr, spoil_bg])
    score = float(np.dot(weights, vec))  # 0..1 higher => more spoofy
    passed = (score < 0.5)

    details = {
        "lip_sync": lip,
        "ocr_hits": ocr_susp,
        "bg_drift": drift,
        "ocr_excerpt": "\n".join(ocr_txt.splitlines()[:10])
    }
    return ModalityResult("video_multicheck", score, details, passed)

In [ ]:
def reflect_and_fix(path, results, attempt=1):
    """
    Simple reflection: detect contradictions and rerun a module with stricter params.
    Returns updated results list and a reflection note.
    """
    note = ""
    # Find audio and video scores if present
    r_audio = next((r for r in results if r.name=="audio_spoof"), None)
    r_video = next((r for r in results if r.name=="video_multicheck"), None)

    # Contradiction check
    if r_audio and r_video:
        # Strong disagreement?
        if (r_audio.score <= 0.2 and r_video.score >= 0.8) or (r_audio.score >= 0.8 and r_video.score <= 0.2):
            note = f"Attempt {attempt}: Contradiction detected. Re-running video with denser sampling and OCR."
            # Re-run video with stricter params
            # (You’d implement alt funcs with step=5, more frames, etc.)
            # For brevity, call the same function; in your code, add params to video_module()
            new_video = video_module(path)
            results = [r for r in results if r.name!="video_multicheck"] + [new_video]
    return results, note

In [ ]:
def run_agentic_checker(path) -> Decision:
    mod = detect_modality(path)
    results = []
    rationale = []

    if mod == "audio":
        results.append(audio_module(path))
    elif mod == "video":
        results.append(video_module(path))
        # Optional: also extract audio track and run audio_module on it for cross-check
        # (left out for brevity)
    else:
        return Decision("unknown", 0.5, "Unsupported modality", [])

    # Reflection loop (one pass demo)
    results, note = reflect_and_fix(path, results, attempt=1)
    if note: rationale.append(note)

    # Final fusion (if multiple results present)
    if len(results) == 1:
        final_score = results[0].score
    else:
        # average or weighted by trust
        final_score = float(np.mean([r.score for r in results]))

    final_label = "spoof" if final_score >= 0.5 else "bonafide"

    # Build explanation
    for r in results:
        if r.name == "audio_spoof":
            rationale.append(f"Audio model spoof_prob={r.score:.2f} ({'suspicious' if r.score>=0.5 else 'clean'}).")
        if r.name == "video_multicheck":
            rationale.append(
                f"Video checks -> lip_sync={r.details['lip_sync']:.2f}, ocr_hits={r.details['ocr_hits']:.2f}, bg_drift={r.details['bg_drift']:.2f}."
            )

    return Decision(final_label, final_score, " ".join(rationale), [asdict(r) for r in results])

# Example:
# d = run_agentic_checker("/content/path/to/video.mp4")
# print(json.dumps(asdict(d), indent=2))

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

# Wrap your xgboost in a calibrator during training:
cal = CalibratedClassifierCV(clf, method="isotonic", cv=5)
cal.fit(Xtr_s, ytr)
proba_cal = cal.predict_proba(Xte_s)[:,1]

In [ ]:
import uuid, datetime, json, os
log_dir = "/content/drive/MyDrive/MUFG SIH/logs"
os.makedirs(log_dir, exist_ok=True)

def log_decision(path, decision: Decision):
    rid = str(uuid.uuid4())[:8]
    now = datetime.datetime.utcnow().isoformat()
    payload = {"ts": now, "file": path, "decision": asdict(decision)}
    with open(os.path.join(log_dir, f"dec_{rid}.json"), "w") as f:
        json.dump(payload, f, indent=2)

In [ ]:
# =========================
# 0) INSTALLS (one-time)
# =========================
!apt-get -qq install -y ffmpeg >/dev/null
!pip -q install praat-parselmouth soundfile librosa tqdm

# =========================
# 1) IMPORTS & CONFIG
# =========================
import os, glob, json, math, subprocess, shlex, warnings
import numpy as np
import pandas as pd
import librosa, librosa.display
import soundfile as sf
import parselmouth
from parselmouth.praat import call
from tqdm import tqdm

warnings.filterwarnings("ignore")

AUDIO_ROOT = "/content/deepfake_audio_dataset"   # <- your unpacked dataset
SR = 16000                                       # work at 16k for uniformity
N_FFT = 1024
HOP = 256

# gather audio files
EXTS = ["wav","mp3","flac","m4a","ogg","aac","wma","pcm"]
audio_files = []
for ext in EXTS:
    audio_files += glob.glob(f"{AUDIO_ROOT}/**/*.{ext}", recursive=True)
print(f"Found {len(audio_files)} files")

# =========================
# 2) UTILS
# =========================
def safe_load(path, sr=SR):
    try:
        y, s = librosa.load(path, sr=sr, mono=True)
        # trim crazy small values
        if len(y)==0:
            return None, sr
        y = np.ascontiguousarray(y)
        return y, s
    except Exception:
        return None, sr

def vad_segments_energy(y, top_db=30, frame_length=2048, hop_length=512):
    """
    Speech segments via librosa.effects.split (energy-based).
    Returns list of (start, end) sample indices for SPEECH.
    """
    if y is None or len(y)==0:
        return []
    intervals = librosa.effects.split(y, top_db=top_db, frame_length=frame_length, hop_length=hop_length)
    return [(int(a), int(b)) for a,b in intervals]

def invert_segments(segments, total_len):
    """ Non-speech = gaps between segments. """
    if total_len <= 0:
        return []
    if not segments:
        return [(0, total_len)]
    out, cur = [], 0
    for a,b in segments:
        if a > cur:
            out.append((cur, a))
        cur = b
    if cur < total_len:
        out.append((cur, total_len))
    return out

def segment_energy(y, segs):
    if not segs:
        return 0.0
    e = []
    for a,b in segs:
        if b>a:
            e.append(np.mean(y[a:b]**2))
    return float(np.mean(e)) if e else 0.0

def snr_db(y, sr=SR):
    """
    Estimate SNR using energy-VAD:
    - speech power = mean energy over speech regions
    - noise power  = mean energy over non-speech regions
    Falls back to percentile-based noise if no non-speech.
    """
    seg_speech = vad_segments_energy(y)
    seg_noise = invert_segments(seg_speech, len(y))

    p_speech = segment_energy(y, seg_speech)
    if seg_noise:
        p_noise = segment_energy(y, seg_noise)
    else:
        # fallback: 10th percentile of frame energies
        S = librosa.feature.rms(y=y, frame_length=2048, hop_length=512).ravel()**2
        p_noise = float(np.percentile(S, 10)) if S.size else 1e-8

    p_speech = max(p_speech, 1e-12)
    p_noise  = max(p_noise, 1e-12)
    return 10*np.log10(p_speech/p_noise), seg_speech, seg_noise

def noise_profile_drift(y, sr, seg_noise, n_fft=N_FFT, hop=HOP):
    """
    Measure how stationary the noise is across non-speech chunks.
    - compute mel spectrogram energy (or linear mag) on noise regions
    - compare chunk spectrums by cosine similarity
    - output 1 - median(similarity) in [0,1] (higher => more drift)
    """
    specs = []
    for a,b in seg_noise:
        if b-a < sr//4:  # need a bit of noise
            continue
        Y = np.abs(librosa.stft(y[a:b], n_fft=n_fft, hop_length=hop)) + 1e-8
        v = np.mean(Y, axis=1)          # average spectrum
        v = v/np.linalg.norm(v)         # normalize
        specs.append(v)
    if len(specs) < 2:
        return 0.0  # not enough noise to compare

    sims = []
    base = specs[0]
    for v in specs[1:]:
        sims.append(float(np.dot(base, v)))
    sims = np.clip(sims, 0.0, 1.0)
    drift = float(1.0 - np.median(sims))
    return drift

def prosody_features(y, sr=SR, hop=HOP):
    """
    Prosody: F0 via YIN, RMS, speaking-rate proxy.
    speaking-rate (proxy): voiced_frames_per_second (rough).
    """
    if len(y) < sr//2:
        return dict(f0_mean=np.nan, f0_std=np.nan, rms_mean=np.nan, rms_std=np.nan, speak_rate=np.nan)

    f0 = librosa.yin(y, fmin=50, fmax=400, sr=sr, frame_length=2048, hop_length=hop)
    f0 = f0[np.isfinite(f0)]
    f0_mean = float(np.mean(f0)) if f0.size else np.nan
    f0_std  = float(np.std(f0))  if f0.size else np.nan

    rms = librosa.feature.rms(y=y, frame_length=2048, hop_length=hop).ravel()
    rms_mean = float(np.mean(rms)) if rms.size else np.nan
    rms_std  = float(np.std(rms))  if rms.size else np.nan

    # speaking-rate proxy: fraction of frames with F0 finite * frames per sec
    frames = max(1, int(len(y)/hop))
    voiced = int((librosa.yin(y, fmin=50, fmax=400, sr=sr, frame_length=2048, hop_length=hop) > 0).sum())
    fps = sr / hop
    speak_rate = float((voiced/frames) * fps) if frames>0 else np.nan

    return dict(f0_mean=f0_mean, f0_std=f0_std, rms_mean=rms_mean, rms_std=rms_std, speak_rate=speak_rate)

def praat_voice_quality(y, sr=SR):
    """
    Jitter / Shimmer / HNR using Praat via parselmouth.
    Returns NaNs if unstable.
    """
    try:
        snd = parselmouth.Sound(y, sr)
        pitch = call(snd, "To Pitch", 0.0, 75, 600)  # typical human range
        point_proc = call(snd, "To PointProcess (periodic, cc)", 75, 600)
        jitter = call(point_proc, "Get jitter (local)", 0, 0, 75, 600, 1.3, 1.6)   # ~ %
        shimmer = call([snd, point_proc], "Get shimmer (local)", 0, 0, 75, 600, 1.3, 1.6, 0.03, 0.45)  # ~ dB-ish
        hnr = call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
        hnr_mean = call(hnr, "Get mean", 0, 0)  # dB
        return dict(jitter_local=float(jitter), shimmer_local=float(shimmer), hnr_mean=float(hnr_mean))
    except Exception:
        return dict(jitter_local=np.nan, shimmer_local=np.nan, hnr_mean=np.nan)

def praat_formants(y, sr=SR):
    """
    Estimate median F1-F3 using Praat Burg method.
    """
    try:
        snd = parselmouth.Sound(y, sr)
        # 5 formants up to 5500 Hz typical; adjust for male/female if needed
        form = call(snd, "To Formant (burg)", 0.025, 5, 5500, 0.025, 50)
        # sample across duration
        T = snd.get_total_duration()
        ts = np.linspace(0.1*T, 0.9*T, 7)
        F1,F2,F3 = [],[],[]
        for t in ts:
            F1.append(call(form, "Get value at time", 1, t, "Hertz", "Linear"))
            F2.append(call(form, "Get value at time", 2, t, "Hertz", "Linear"))
            F3.append(call(form, "Get value at time", 3, t, "Hertz", "Linear"))
        F1 = np.array([f for f in F1 if np.isfinite(f)])
        F2 = np.array([f for f in F2 if np.isfinite(f)])
        F3 = np.array([f for f in F3 if np.isfinite(f)])
        return dict(F1_med=float(np.median(F1)) if F1.size else np.nan,
                    F2_med=float(np.median(F2)) if F2.size else np.nan,
                    F3_med=float(np.median(F3)) if F3.size else np.nan)
    except Exception:
        return dict(F1_med=np.nan, F2_med=np.nan, F3_med=np.nan)

def phase_artifact_scores(y, sr=SR, n_fft=N_FFT, hop=HOP):
    """
    Simple phase/vocoder artifact proxies:
    - mean absolute phase frame-diff (phase discontinuity)
    - spectral flatness (avg)
    """
    if len(y) < sr//2:
        return dict(phase_disc=np.nan, spec_flatness=np.nan)

    S = librosa.stft(y, n_fft=n_fft, hop_length=hop)
    phase = np.angle(S)
    # frame-to-frame phase difference magnitude
    dp = np.abs(np.diff(phase, axis=1))
    phase_disc = float(np.mean(dp))

    flat = librosa.feature.spectral_flatness(S=np.abs(S)).ravel()
    spec_flatness = float(np.mean(flat)) if flat.size else np.nan
    return dict(phase_disc=phase_disc, spec_flatness=spec_flatness)

def rolloff_lowpass_hz(y, sr=SR):
    """ crude low-pass cutoff estimator via 95% spectral rolloff """
    ro = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.95).ravel()
    return float(np.nanmedian(ro)) if ro.size else np.nan

def ffprobe_codec(path):
    """
    Try to parse codec & bitrate via ffprobe. Returns dict with codec_name, bit_rate (bps).
    If ffprobe fails, returns NaNs.
    """
    try:
        cmd = f'ffprobe -v error -select_streams a:0 -show_entries stream=codec_name,bit_rate -of json "{path}"'
        out = subprocess.check_output(shlex.split(cmd), stderr=subprocess.STDOUT, timeout=10)
        data = json.loads(out.decode("utf-8"))
        streams = data.get("streams", [])
        if not streams:
            return dict(codec_name=np.nan, bit_rate=np.nan)
        s0 = streams[0]
        return dict(codec_name=s0.get("codec_name", np.nan),
                    bit_rate=float(s0.get("bit_rate")) if s0.get("bit_rate") else np.nan)
    except Exception:
        return dict(codec_name=np.nan, bit_rate=np.nan)

# =========================
# 3) FEATURE EXTRACTION LOOP
# =========================
rows = []
for f in tqdm(audio_files):
    y, sr = safe_load(f, SR)
    if y is None or len(y) < SR//2:
        rows.append(dict(
            file=os.path.basename(f), rel_path=os.path.relpath(f, AUDIO_ROOT),
            snr_db=np.nan, noise_drift=np.nan,
            f0_mean=np.nan, f0_std=np.nan, rms_mean=np.nan, rms_std=np.nan, speak_rate=np.nan,
            jitter_local=np.nan, shimmer_local=np.nan, hnr_mean=np.nan,
            F1_med=np.nan, F2_med=np.nan, F3_med=np.nan,
            phase_disc=np.nan, spec_flatness=np.nan,
            rolloff95_hz=np.nan, codec_name=np.nan, bit_rate=np.nan
        ))
        continue

    # SNR + noise drift
    snr, seg_speech, seg_noise = snr_db(y, sr)
    drift = noise_profile_drift(y, sr, seg_noise)

    # Prosody
    pros = prosody_features(y, sr)

    # Voice quality
    vq = praat_voice_quality(y, sr)

    # Formants
    fm = praat_formants(y, sr)

    # Phase/vocoder proxies
    ph = phase_artifact_scores(y, sr)

    # Low-pass & codec
    rolloff = rolloff_lowpass_hz(y, sr)
    codec = ffprobe_codec(f)

    rows.append(dict(
        file=os.path.basename(f),
        rel_path=os.path.relpath(f, AUDIO_ROOT),
        snr_db=float(snr),
        noise_drift=float(drift),
        **pros, **vq, **fm, **ph,
        rolloff95_hz=float(rolloff),
        **codec
    ))

df_audio_feats = pd.DataFrame(rows)
print(df_audio_feats.shape)
df_audio_feats.head()

# =========================
# 4) SAVE FEATURES
# =========================
out_dir = "/content/drive/MyDrive/processed"
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "audio_forensics_features.csv")
df_audio_feats.to_csv(out_path, index=False)
print("Saved:", out_path)

# =========================
# 5) (OPTIONAL) MERGE WITH YOUR MFCC CSV
# =========================
# If you already created /content/drive/MyDrive/processed/processed_audio_mfcc.csv
mfcc_path = "/content/drive/MyDrive/processed/processed_audio_mfcc.csv"
if os.path.exists(mfcc_path):
    df_mfcc = pd.read_csv(mfcc_path)
    # join on rel_path (recommended); fallback to file name if needed
    on_keys = "rel_path" if "rel_path" in df_mfcc.columns else "file"
    df_merged = pd.merge(df_mfcc, df_audio_feats, on=on_keys, how="left")
    merged_out = os.path.join(out_dir, "audio_features_merged.csv")
    df_merged.to_csv(merged_out, index=False)
    print("Merged & saved:", merged_out)
else:
    print("MFCC CSV not found at:", mfcc_path, "— skipping merge.")

Found 0 files


0it [00:00, ?it/s]

(0, 0)
Saved: /content/drive/MyDrive/processed/audio_forensics_features.csv
MFCC CSV not found at: /content/drive/MyDrive/processed/processed_audio_mfcc.csv — skipping merge.


In [ ]:
import glob

# Search entire Drive for audio
files = glob.glob("/content/drive/MyDrive/**/*.wav", recursive=True)
print("Found wav:", len(files))
print(files[:10])

files_mp3 = glob.glob("/content/drive/MyDrive/**/*.mp3", recursive=True)
print("Found mp3:", len(files_mp3))
print(files_mp3[:10])

Found wav: 14106
['/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file1723.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file1818.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file2192.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file961.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file2018.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file1340.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file2099.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfak

In [ ]:




import os

for root, dirs, files in os.walk("/content/deepfake_audio_dataset"):
    print(root, len(files), files[:5])

In [ ]:
# ---- CONFIG (point to the ACTUAL dataset) ----
AUDIO_ROOT = "/content/drive/MyDrive/Deepfake_audio"   # <-- FIXED
SR = 16000

# Quick sanity check
import os, glob
wav_files = glob.glob(f"{AUDIO_ROOT}/**/*.wav", recursive=True)
print("Found wav:", len(wav_files))
print("Example:", wav_files[:3])

Found wav: 14106
Example: ['/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file1723.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file1818.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', '/content/drive/MyDrive/Deepfake_audio/for-2sec/for-2seconds/testing/real/file2192.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav']


In [ ]:
# ---- ONE-TIME INSTALLS ----
!apt-get -qq install -y ffmpeg >/dev/null
!pip -q install praat-parselmouth soundfile librosa tqdm

In [ ]:
# ---- IMPORTS & HELPERS ----
import os, glob, json, math, subprocess, shlex, warnings
import numpy as np
import pandas as pd
import librosa, soundfile as sf
import parselmouth
from parselmouth.praat import call
from tqdm import tqdm

warnings.filterwarnings("ignore")
N_FFT = 1024
HOP = 256

def safe_load(path, sr=SR):
    try:
        y, s = librosa.load(path, sr=sr, mono=True)
        if len(y)==0: return None, sr
        return np.ascontiguousarray(y), s
    except Exception:
        return None, sr

def vad_segments_energy(y, top_db=30, frame_length=2048, hop_length=512):
    if y is None or len(y)==0: return []
    intervals = librosa.effects.split(y, top_db=top_db, frame_length=frame_length, hop_length=hop_length)
    return [(int(a), int(b)) for a,b in intervals]

def invert_segments(segments, total_len):
    if total_len <= 0: return []
    if not segments: return [(0, total_len)]
    out, cur = [], 0
    for a,b in segments:
        if a > cur: out.append((cur, a))
        cur = b
    if cur < total_len: out.append((cur, total_len))
    return out

def segment_energy(y, segs):
    if not segs: return 0.0
    e = [np.mean(y[a:b]**2) for a,b in segs if b>a]
    return float(np.mean(e)) if e else 0.0

def snr_db(y, sr=SR):
    seg_speech = vad_segments_energy(y)
    seg_noise  = invert_segments(seg_speech, len(y))
    p_speech = max(segment_energy(y, seg_speech), 1e-12)
    if seg_noise:
        p_noise = segment_energy(y, seg_noise)
    else:
        S = librosa.feature.rms(y=y, frame_length=2048, hop_length=512).ravel()**2
        p_noise = float(np.percentile(S,10)) if S.size else 1e-8
    p_noise = max(p_noise, 1e-12)
    return 10*np.log10(p_speech/p_noise), seg_speech, seg_noise

def noise_profile_drift(y, sr, seg_noise, n_fft=N_FFT, hop=HOP):
    specs=[]
    for a,b in seg_noise:
        if b-a < sr//4: continue
        Y = np.abs(librosa.stft(y[a:b], n_fft=n_fft, hop_length=hop))+1e-8
        v = np.mean(Y, axis=1)
        v = v/np.linalg.norm(v)
        specs.append(v)
    if len(specs) < 2: return 0.0
    sims = [float(np.dot(specs[0], v)) for v in specs[1:]]
    sims = np.clip(sims, 0.0, 1.0)
    return float(1.0 - np.median(sims))

def prosody_features(y, sr=SR, hop=HOP):
    if len(y) < sr//2:
        return dict(f0_mean=np.nan,f0_std=np.nan,rms_mean=np.nan,rms_std=np.nan,speak_rate=np.nan)
    f0_all = librosa.yin(y, fmin=50, fmax=400, sr=sr, frame_length=2048, hop_length=hop)
    f0 = f0_all[np.isfinite(f0_all)]
    f0_mean = float(np.mean(f0)) if f0.size else np.nan
    f0_std  = float(np.std(f0))  if f0.size else np.nan
    rms = librosa.feature.rms(y=y, frame_length=2048, hop_length=hop).ravel()
    rms_mean = float(np.mean(rms)) if rms.size else np.nan
    rms_std  = float(np.std(rms))  if rms.size else np.nan
    frames = max(1, int(len(y)/hop))
    voiced = int((f0_all>0).sum())
    fps = sr / hop
    speak_rate = float((voiced/frames) * fps) if frames>0 else np.nan
    return dict(f0_mean=f0_mean,f0_std=f0_std,rms_mean=rms_mean,rms_std=rms_std,speak_rate=speak_rate)

def praat_voice_quality(y, sr=SR):
    try:
        snd = parselmouth.Sound(y, sr)
        point_proc = call(snd, "To PointProcess (periodic, cc)", 75, 600)
        jitter = call(point_proc, "Get jitter (local)", 0, 0, 75, 600, 1.3, 1.6)
        shimmer = call([snd, point_proc], "Get shimmer (local)", 0, 0, 75, 600, 1.3, 1.6, 0.03, 0.45)
        hnr = call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
        hnr_mean = call(hnr, "Get mean", 0, 0)
        return dict(jitter_local=float(jitter), shimmer_local=float(shimmer), hnr_mean=float(hnr_mean))
    except Exception:
        return dict(jitter_local=np.nan, shimmer_local=np.nan, hnr_mean=np.nan)

def praat_formants(y, sr=SR):
    try:
        snd = parselmouth.Sound(y, sr)
        form = call(snd, "To Formant (burg)", 0.025, 5, 5500, 0.025, 50)
        T = snd.get_total_duration()
        ts = np.linspace(0.1*T, 0.9*T, 7)
        vals = []
        for t in ts:
            f1 = call(form, "Get value at time", 1, t, "Hertz", "Linear")
            f2 = call(form, "Get value at time", 2, t, "Hertz", "Linear")
            f3 = call(form, "Get value at time", 3, t, "Hertz", "Linear")
            if np.isfinite(f1) and np.isfinite(f2) and np.isfinite(f3):
                vals.append((f1,f2,f3))
        if not vals: return dict(F1_med=np.nan,F2_med=np.nan,F3_med=np.nan)
        F1,F2,F3 = map(np.median, zip(*vals))
        return dict(F1_med=float(F1),F2_med=float(F2),F3_med=float(F3))
    except Exception:
        return dict(F1_med=np.nan,F2_med=np.nan,F3_med=np.nan)

def phase_artifact_scores(y, sr=SR, n_fft=N_FFT, hop=HOP):
    if len(y) < sr//2: return dict(phase_disc=np.nan, spec_flatness=np.nan)
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop)
    phase = np.angle(S)
    dp = np.abs(np.diff(phase, axis=1))
    phase_disc = float(np.mean(dp))
    flat = librosa.feature.spectral_flatness(S=np.abs(S)).ravel()
    spec_flatness = float(np.mean(flat)) if flat.size else np.nan
    return dict(phase_disc=phase_disc, spec_flatness=spec_flatness)

def rolloff_lowpass_hz(y, sr=SR):
    ro = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.95).ravel()
    return float(np.nanmedian(ro)) if ro.size else np.nan

def ffprobe_codec(path):
    try:
        cmd = f'ffprobe -v error -select_streams a:0 -show_entries stream=codec_name,bit_rate -of json "{path}"'
        out = subprocess.check_output(shlex.split(cmd), stderr=subprocess.STDOUT, timeout=10)
        data = json.loads(out.decode("utf-8"))
        streams = data.get("streams", [])
        if not streams: return dict(codec_name=np.nan, bit_rate=np.nan)
        s0 = streams[0]
        return dict(codec_name=s0.get("codec_name", np.nan),
                    bit_rate=float(s0.get("bit_rate")) if s0.get("bit_rate") else np.nan)
    except Exception:
        return dict(codec_name=np.nan, bit_rate=np.nan)

def subset_from_path(path: str) -> str:
    # assumes: Deepfake_audio/<subset>/.../file.wav
    rel = os.path.relpath(path, AUDIO_ROOT)
    parts = rel.split(os.sep)
    return parts[0] if len(parts) > 1 else "root"

In [ ]:
# ---- FEATURE EXTRACTION ----
rows = []
wav_files = glob.glob(f"{AUDIO_ROOT}/**/*.wav", recursive=True)
print("Total .wav to process:", len(wav_files))

for f in tqdm(wav_files):
    y, sr = safe_load(f, SR)
    if y is None or len(y) < SR//2:
        rows.append(dict(file=os.path.basename(f),
                         rel_path=os.path.relpath(f, AUDIO_ROOT),
                         subset=subset_from_path(f),
                         snr_db=np.nan, noise_drift=np.nan,
                         f0_mean=np.nan,f0_std=np.nan,rms_mean=np.nan,rms_std=np.nan,speak_rate=np.nan,
                         jitter_local=np.nan, shimmer_local=np.nan, hnr_mean=np.nan,
                         F1_med=np.nan,F2_med=np.nan,F3_med=np.nan,
                         phase_disc=np.nan, spec_flatness=np.nan,
                         rolloff95_hz=np.nan, codec_name=np.nan, bit_rate=np.nan))
        continue

    snr, seg_speech, seg_noise = snr_db(y, sr)
    drift = noise_profile_drift(y, sr, seg_noise)
    pros  = prosody_features(y, sr)
    vq    = praat_voice_quality(y, sr)
    fm    = praat_formants(y, sr)
    ph    = phase_artifact_scores(y, sr)
    roll  = rolloff_lowpass_hz(y, sr)
    codec = ffprobe_codec(f)

    rows.append(dict(file=os.path.basename(f),
                     rel_path=os.path.relpath(f, AUDIO_ROOT),
                     subset=subset_from_path(f),
                     snr_db=float(snr), noise_drift=float(drift),
                     **pros, **vq, **fm, **ph,
                     rolloff95_hz=float(roll), **codec))

df_audio_feats = pd.DataFrame(rows)
print(df_audio_feats.shape)
df_audio_feats.head()

Total .wav to process: 14106


100%|██████████| 14106/14106 [54:13<00:00,  4.34it/s]


(14106, 21)


,file,rel_path,subset,snr_db,noise_drift,f0_mean,f0_std,rms_mean,rms_std,speak_rate,...,shimmer_local,hnr_mean,F1_med,F2_med,F3_med,phase_disc,spec_flatness,rolloff95_hz,codec_name,bit_rate
0,file1723.wav_16k.wav_norm.wav_mono.wav_silence...,for-2sec/for-2seconds/testing/real/file1723.wa...,for-2sec,10.946198,0.0,132.035078,40.930680,0.127294,0.065641,63.0,...,NaN,NaN,523.805702,2158.299723,3054.818941,2.091907,0.038867,4101.5625,pcm_s16le,256000.0
1,file1818.wav_16k.wav_norm.wav_mono.wav_silence...,for-2sec/for-2seconds/testing/real/file1818.wa...,for-2sec,28.471113,0.0,123.877050,30.470802,0.139609,0.070590,63.0,...,NaN,NaN,726.562943,1804.250026,2796.229949,2.081776,0.041340,5140.6250,pcm_s16le,256000.0
2,file2192.wav_16k.wav_norm.wav_mono.wav_silence...,for-2sec/for-2seconds/testing/real/file2192.wa...,for-2sec,41.155881,0.0,141.718516,59.693975,0.104777,0.059240,63.0,...,NaN,NaN,451.384115,1772.611387,2861.925389,2.093791,0.017849,4093.7500,pcm_s16le,256000.0
3,file961.wav_16k.wav_norm.wav_mono.wav_silence....,for-2sec/for-2seconds/testing/real/file961.wav...,for-2sec,30.792879,0.0,198.165947,51.047331,0.169630,0.078919,63.0,...,NaN,NaN,473.915378,2132.903171,3188.873791,2.091629,0.040176,5218.7500,pcm_s16le,256000.0
4,file2018.wav_16k.wav_norm.wav_mono.wav_silence...,for-2sec/for-2seconds/testing/real/file2018.wa...,for-2sec,12.087560,0.0,123.181972,41.303281,0.141617,0.077163,63.0,...,NaN,NaN,476.173752,1836.125622,2818.524685,2.075680,0.023950,4468.7500,pcm_s16le,256000.0


In [ ]:
# ---- SAVE + MERGE WITH MFCC CSV (if present) ----
out_dir = "/content/drive/MyDrive/processed"
os.makedirs(out_dir, exist_ok=True)
forensic_path = os.path.join(out_dir, "audio_forensics_features.csv")
df_audio_feats.to_csv(forensic_path, index=False)
print("Saved:", forensic_path)

mfcc_path = "/content/drive/MyDrive/processed/processed_audio_mfcc.csv"
if os.path.exists(mfcc_path):
    df_mfcc = pd.read_csv(mfcc_path)
    key = "rel_path" if "rel_path" in df_mfcc.columns else "file"
    merged = pd.merge(df_mfcc, df_audio_feats, on=key, how="left")
    merged_out = os.path.join(out_dir, "audio_features_merged.csv")
    merged.to_csv(merged_out, index=False)
    print("Merged & saved:", merged_out)
else:
    print("MFCC CSV not found at:", mfcc_path, "— skipping merge.")

Saved: /content/drive/MyDrive/processed/audio_forensics_features.csv
MFCC CSV not found at: /content/drive/MyDrive/processed/processed_audio_mfcc.csv — skipping merge.


In [ ]:
# --- Auto-find MFCC CSV anywhere in Drive and merge ---

import os, glob, pandas as pd

# 1) Find the MFCC CSV (handles duplicates like 'processed_audio_mfcc (1).csv')
candidates = glob.glob("/content/drive/MyDrive/**/processed_audio_mfcc*.csv", recursive=True)
print("Found MFCC candidates:", candidates)

if not candidates:
    raise FileNotFoundError("Could not find any 'processed_audio_mfcc*.csv' in MyDrive. "
                            "If you know the path, paste it into `mfcc_path` manually.")

# 2) Pick the most recently modified file
mfcc_path = max(candidates, key=os.path.getmtime)
print("Using MFCC file:", mfcc_path)

# 3) Load both tables
forensic_path = "/content/drive/MyDrive/processed/audio_forensics_features.csv"
df_forensic = pd.read_csv(forensic_path)
df_mfcc = pd.read_csv(mfcc_path)

# 4) Choose join key (prefer rel_path if present)
key = "rel_path" if "rel_path" in df_mfcc.columns and "rel_path" in df_forensic.columns else "file"
if key == "file":
    # Optional: sanity log if rel_path missing in one side
    missing_cols = [c for c in ["rel_path"] if c not in df_mfcc.columns or c not in df_forensic.columns]
    print(f"Joining on 'file' (missing {missing_cols})")

# 5) Merge
df_merged = pd.merge(df_mfcc, df_forensic, on=key, how="left")

# 6) Save to a stable location
out_dir = "/content/drive/MyDrive/processed"
os.makedirs(out_dir, exist_ok=True)
merged_out = os.path.join(out_dir, "audio_features_merged.csv")
df_merged.to_csv(merged_out, index=False)
print("Merged & saved:", merged_out)

# 7) Quick peek
print(df_merged.shape)
df_merged.head()

Found MFCC candidates: ['/content/drive/MyDrive/MUFG SIH/processed_audio_mfcc.csv']
Using MFCC file: /content/drive/MyDrive/MUFG SIH/processed_audio_mfcc.csv
Merged & saved: /content/drive/MyDrive/processed/audio_features_merged.csv
(163934, 36)


,file_x,rel_path,subset_x,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,...,shimmer_local,hnr_mean,F1_med,F2_med,F3_med,phase_disc,spec_flatness,rolloff95_hz,codec_name,bit_rate
0,file2107.wav,for-original/for-original/testing/real/file210...,for-original,-256.34973,118.939630,10.461822,39.780823,-19.730724,1.610053,0.178933,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,file447.wav,for-original/for-original/testing/real/file447...,for-original,-261.06073,56.958260,0.395447,27.690630,-16.021925,2.031633,-11.167014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,file125.wav,for-original/for-original/testing/real/file125...,for-original,-251.60540,75.884340,11.717958,38.722553,-14.731581,1.523854,-5.391180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,file1784.wav,for-original/for-original/testing/real/file178...,for-original,-207.51080,95.789400,11.629858,50.834805,-19.259535,4.548469,-9.069361,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,file1360.wav,for-original/for-original/testing/real/file136...,for-original,-217.65309,94.334496,1.279012,18.014864,-23.635376,10.071319,-20.874025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np

PATH = "/content/drive/MyDrive/processed/audio_features_merged.csv"
df = pd.read_csv(PATH)
print(df.shape); df.head()

# Label from subset: for-original => bonafide, else spoof
SPOOF_FOLDERS = {"for-norm","for-rerec","for-2sec"}
if "subset" not in df.columns:
    # fallback from rel_path if needed
    def subset_from_rel(p):
        try: return p.split("/")[0]
        except: return "unknown"
    df["subset"] = df["rel_path"].apply(subset_from_rel)

df["label"] = np.where(df["subset"]=="for-original", "bonafide",
                 np.where(df["subset"].isin(SPOOF_FOLDERS), "spoof", "unknown"))
df = df[df["label"]!="unknown"].copy()
print(df["label"].value_counts())

# Pick features (forensics + MFCCs). Keep only those present.
maybe = [
  # MFCCs
  *[f"mfcc_{i}" for i in range(1,14)],
  # forensics
  "snr_db","noise_drift","f0_mean","f0_std","rms_mean","rms_std","speak_rate",
  "jitter_local","shimmer_local","hnr_mean",
  "F1_med","F2_med","F3_med",
  "phase_disc","spec_flatness",
  "rolloff95_hz"
]
feat_cols = [c for c in maybe if c in df.columns]
print("Using", len(feat_cols), "features")

(163934, 36)
label
spoof       95209
bonafide    68725
Name: count, dtype: int64
Using 29 features
